In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM4_ESM.xlsx', sheet_name='1K')

In [5]:
data_df = data_df.set_index('Unnamed: 0')

In [102]:
data_df

,HFD,HFD.1,HFD.2,HFD.3,HFD.4,HFD.5,HFD.6,HFD.7,HFD.8,HFD.9,...,HFD+SANA .2,HFD+SANA .3,HFD+SANA .4,HFD+SANA .5,HFD+SANA .6,HFD+SANA .7,HFD+SANA .8,HFD+SANA .9,HFD+SANA .10,HFD+SANA .11
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
iScap_BAT,0.56,0.57,0.53,0.65,0.4801,0.3616,0.6393,0.5813,0.6533,0.525,...,0.43,0.30,0.3678,NaN,0.3668,0.4047,0.5179,0.4382,0.5938,0.3856
i_WAT,2.01,1.79,1.90,1.80,4.4360,4.1530,4.8620,5.2020,4.0040,4.293,...,0.92,1.01,0.3260,1.453,1.5890,1.1250,1.3730,2.5000,0.9930,3.1720
sScap_WAT,2.12,1.32,1.33,1.32,1.1850,1.6700,2.3110,2.0430,3.7690,2.875,...,0.89,0.81,0.3290,0.909,1.2830,0.6260,0.8320,1.2930,0.7720,1.3880
p_WAT,4.43,6.21,5.24,5.94,4.6750,3.2750,3.2190,5.1320,4.1880,3.889,...,2.89,1.70,0.6610,1.883,3.1260,2.3210,2.4370,NaN,2.2050,NaN


In [10]:
hfd = ['HFD', 'HFD.1', 'HFD.2', 'HFD.3', 'HFD.4', 'HFD.5', 'HFD.6', 'HFD.7',
       'HFD.8', 'HFD.9']
hfd_sal = ['HFD+SAL ', 'HFD+SAL .1', 'HFD+SAL .2', 'HFD+SAL .3',
       'HFD+SAL .4', 'HFD+SAL .5', 'HFD+SAL .6', 'HFD+SAL .7', 'HFD+SAL .8',
       'HFD+SAL .9', 'HFD+SAL .10', 'HFD+SAL .11', 'HFD+SAL .12',
       'HFD+SAL .13']
hfd_sana = ['HFD+SANA ', 'HFD+SANA .1', 'HFD+SANA .2', 'HFD+SANA .3',
       'HFD+SANA .4', 'HFD+SANA .5', 'HFD+SANA .6', 'HFD+SANA .7',
       'HFD+SANA .8', 'HFD+SANA .9', 'HFD+SANA .10', 'HFD+SANA .11']

In [103]:
for cat in data_df.index:
    print(cat)
    grp1, grp2, grp3 = data_df[hfd].loc[cat].dropna().values, \
                        data_df[hfd_sal].loc[cat].dropna().values, \
                        data_df[hfd_sana].loc[cat].dropna().values

    collect = [grp1, grp2, grp3]
    print('N samples: ' + str(len(np.concatenate([grp1, grp2, grp3]))))
    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3)
    print('F(' + str(len(np.concatenate(collect))-3) + ', 2) = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate([grp1, grp2, grp3]), 
                                      groups=np.concatenate([['HFD']*len(grp1), 
                                                             ['HFD+SAL']*len(grp2), 
                                                             ['HFD+SANA']*len(grp3)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)


    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*[grp1, grp2, grp3]).pvalue
    tukey_df = pd.DataFrame({'g1':['HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[1][2]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())

iScap_BAT
N samples: 35
ANOVA:
F(32, 2) = 6.663273577052259, p = 0.0038085307013093826
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |           0.0922652  |      0.0647224  |
| HFD vs HFD+SANA     |                     |           0.00319725 |      0.00274803 |
| HFD+SAL vs HFD+SANA |                     |           0.456125   |      0.289784   |
i_WAT 
N samples: 36
ANOVA:
F(33, 2) = 10.939100765687241, p = 0.0002266637915246635
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |          0.651085    |     0.343131    |
| HFD vs HFD+SANA     |                     |          0.000762237 |     0.000243699 |
| HFD+SAL vs HFD+SANA |                     |

Here is Figure 1K:

![file](../img/pub_fig_1k.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. I'll assume that a one-way ANOVA was performed for each group within each of the four tissues shown. The p-values are also labeled somewhat ambiguously. I'll assume that the horizontally-oriented p-values represent the comparison HFD+SAL vs HFD+SANA, the left vertically-oriented p-values represent the comparison HFD vs HFD+SAL and the right vertically-oriented p-values represent the comparison HFD vs HFD+SANA. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38) for this figure, most of these p-values are incorrect both for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test) and for Tukey's post-hoc test. I've compiled summary stats and tables for each tissue tested:

### iScap_BAT

N samples: 35
ANOVA:
F(32, 2) = 6.663273577052259, p = 0.0038085307013093826

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |          0.08           |           0.0923  |      0.0647  |
| HFD vs HFD+SANA     |             0.004        |           0.00320 |      0.00275 |
| HFD+SAL vs HFD+SANA |               0.3      |           0.456   |      0.290   |

### i_WAT 
N samples: 36
ANOVA:
F(33, 2) = 10.939100765687241, p = 0.0002266637915246635

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |           0.2          |          0.651    |     0.343    |
| HFD vs HFD+SANA     |             0.0001        |          0.000762 |     0.000244 |
| HFD+SAL vs HFD+SANA |          0.005           |          0.00210  |     0.00515  |

### sScap_WAT
N samples: 36
ANOVA:
F(33, 2) = 9.907279941382573, p = 0.00042662326451083635

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |               0.5      |          0.891    |     0.431    |
| HFD vs HFD+SANA     |           0.001          |          0.000983 |     0.000494 |
| HFD+SAL vs HFD+SANA |          0.008           |          0.00176  |     0.00659  |

### p_WAT
N samples: 33
ANOVA:
F(30, 2) = 23.63878917987882, p = 6.855138887436469e-07

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |              0.7       |          1.00           |     0.595     |
| HFD vs HFD+SANA     |                <0.0001     |          3.55E-5 |     3.42E-5 |
| HFD+SAL vs HFD+SANA |           0.00007 (7E-5)          |          4.58E-06 |     8.81E-7 |

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).